In [3]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R


def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])
    j = tf.index(0, [m])

    with tf.loop(n-1) as i:
        R[i, i] = tf.norm(A[j, i])
        Q[j, i] = A[j, i] / R[i, i]

        p, k = tf.index_grid([0, i + 1], [m, n])
        t, = tf.index_grid([i+1], [n])
        R[i, t] = tf.sum(Q[p, i] * A[p, k], axis=0)
        A[p, k] -= Q[p, i] * R[i, k]

    R[n-1, n-1] = tf.norm(A[j, n-1])
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)

QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 444
  IR Compile time: 4.008300 ms
  Compiler time: 1438.007568 ms



In [4]:
#kernels = tf.get_all_generated_kernels()
#for kernel in kernels:
#    print(kernel)

In [5]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 1.9229626863835638e-16
Error using TensorFrost: 1.4379632869159912e-07
Q:
 [[ 0.38164046  0.4589707  -0.21670018  0.6375303  -0.436225  ]
 [ 0.10536594  0.48427615  0.83494586 -0.20836361 -0.11757868]
 [ 0.6480063  -0.27790347  0.2384919   0.30108896  0.5960891 ]
 [ 0.07976632  0.6735651  -0.39972037 -0.29991347  0.53872347]
 [ 0.6457351  -0.15460339 -0.19812036 -0.60789555 -0.38773447]]
R:
 [[ 1.5277998   1.5553625   0.9381008   1.3711534   1.3707838 ]
 [ 0.          0.6546776   0.641195    0.10939406  0.32050806]
 [ 0.          0.          0.64566505  0.08722901 -0.00254943]
 [ 0.          0.          0.          0.08293349 -0.15995835]
 [ 0.          0.          0.          0.          0.16902758]]


In [6]:
#performance test
import time
A = np.random.rand(5000, 5000).astype(np.float32)

#naive NumPy QR decomposition
#start = time.time()
#Q, R = modified_gram_schmidt(A)
#print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
Qtf = Qtf.numpy
print("Time for TensorFrost QR decomposition:", time.time() - start)
Rtf = Rtf.numpy
#householder QR decomposition
#start = time.time()
#Q, R = qr_decomposition(A)
#print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf, Rtf)))

Time for TensorFrost QR decomposition: 5.670893907546997
Time for built-in NumPy QR decomposition: 4.237346887588501
Error: 0.0012285828
Error using TensorFrost: 0.0018776024
